In [614]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from Utils import ErrorMetricsUtils as err
from Utils import CorrectnessMetricUtils as cmu
from Utils import AuxUtils as auxu
from sklearn.preprocessing import StandardScaler
from csv import reader
import random


class LinearModel():
    
    def __init__ (self, loss = "rmse", model_type = "linear", convergence = "Stochastic Gradiet Descent", 
                  penalty = "None", include_bias = True, regression_degree = 2, standarsize = True, normalize = True, 
                  learning_type = "constant", max_iter = float("inf"), alpha = 0.001, epsilon = 0.1):
        self.alpha = alpha
        self.loss = loss
        self.epsilon = epsilon
        self.learning_type = learning_type
        self.theta = None
        self.poly = auxu.PolynomialKernel(regression_degree, include_bias)
        self.fitted = False
        self.model_type = model_type
        self.g = transform_fn_g()
        self.convergence = convergence
        self.max_iter = max_iter
    
    def call_convergence ():
        if (self.convergence == "Stochatic Gradient Descent"):
            sch_grad_descent (Xk_train, y_train)
        elif (self.convergence == "Normal Equations"):
            normal_eqn (Xk_train, y_train)
        elif (self.convergence == "Newton's Method"):
             newton_method (Xk_train, y_train)
        else:
            bch_grad_descent (Xk_train, y_train)
        return
    
    # General linear model
    def fit (self, train_set):
        """
        ToDo
        Is this generalizable?
        Is it possible for Xk_test to be one dimensional?
        Should we heck for X_train being one dimensional and fix?
        """
        X_train = np.asarray(train_set)[:, :-1]
        y_train = np.asarray(train_set)[:, -1]
        if (self.standardize):
            X_train = auxu.standardize(X_train)
        if (self.normalize):
            X_train = auxu.normalize(X_train)
        Xk_train = self.poly.kernelize(X_train)
        self.theta = np.zeros(Xk_train.shape[1])
        self.call_convergence()
        self.fitted = True
        return self.theta
    
    def predict (self, test_set):
        """
        ToDo
        Is this generalizable?
        Is it possible for Xk_test to be one dimensional?
        Should we heck for X_train being one dimensional and fix?
        """
        X_test = np.asarray(test_set)[:, :-1]
        if (self.standardize):
            X_test = auxu.standardize(X_test)
        if (self.normalize):
            X_test = auxu.normalize(X_test)
        Xk_test = self.poly.kernelize(X_test)
        if (self.check_fitted(Xk_test)): 
            return self.g(Xk_test @ self.theta)
        else: 
            return -1 # some error statement
    
    def check_fitted (self, Xk_test):
        """
        ToDO
        Is this generalizable?
        Is it possible for Xk_test to be one dimensional?
        Should we heck for X_train being one dimensional and fix?
        """
        return self.fitted and self.theta.shape[0] == Xk_test.shape[1]
    
    def calc_loss (self, y, hypo):
        if (self.loss == "rmse"):
            return err.rmse_calc (y, hypo)
        elif (self.loss == "mae"):
            return err.mae_calc (y, hypo)
        elif (self.loss == "kld"):
            return err.kl_divergence_calc (y, hypo)
        elif (self.loss == "cross"):
            return err.cross_entropy_calc (y, hypo)
        else:
            return err.mse_calc (y, hypo)
    
    def neg_gradient (self, X, y, hypo):
        """
        ToDo
        Loss Functions not included
        What about 0-1 and other functions useful for classifiers?
        
        """
        if (self.loss == "rmse"):
            rmse_err = err.rmse_calc (y, hypo)
            return (X.T)@(y-hypo)/rmse_err
        elif (self.loss == "mae"):
            #ToDo
            return
        elif (self.loss == "kld"):
            #ToDo
            return
        elif (self.loss == "cross"):
            #ToDo
            return
        else:
            return (X.T)@(y-hypo)
    
    def transform_fn_g (self):
        """
        ToDo
        Other generalzations
        What needs to cahnge for Locally Weighted Regression?
        """
        if (self.model_type == "perceptron"):
            return lambda x: np.piecewise(x, [x < 0, x >= 0], [0, 1])
        elif (self.model_type == "logistic"):
            return lambda x: 1 / (1 + np.exp(-x))
        else:
            return lambda x:x
        
    def descent_step (neg_gradient):
        """
        ToDo
        Optimal Leerning Rate
        Should we add different types?
        Check different norm orders and how that matters
        """
        if (self.learning_type == "normalized"):
            beta = self.alpha/np.linalg.norm(neg_gradient)
        elif (self.learning_type == "constant"):
            beta = self.alpha
        else:
            # optimal learning type
            # ToDo
            beta = self.alpha
        self.theta = self.theta + beta*neg_gradient
        return np.linalg.norm(beta*neg_gradient)
        
    # General Batch Gradient Descent
    def bch_gradient_descent (self, X, y):
        """
        Add Regularizer
        """
        g = transform_fn_g()
        iters = 0
        while (iters < self.max_iter):
            hypo = g(X @ self.theta)
            neg_gradient = self.neg_gradient(X, y, hypo)
            change_norm = self.descent_step (neg_gradient)
            err = self.calc_loss(y, hypo)
            if (change_norm < self.epsilon): 
                break
            iters = iters + 1
        return
    
    def get_batch (self, X, y, batch_size):
        """
        ToDo - more randomization?
            1. system time as seed
            2. all elements randomized
        """
        random.seed(3)
        index = random.randrange(X.shape[0] - batch_size)
        return X[index:index+batch_size, :], y[index:index+batch_size]
    
    # General Stochastic Gradient Descent
    def sch_grad_descent (self, X, y):
        """
        ToDo
        Add Regularizer
        """
        batch_size = 49
        g = transform_fn_g() 
        iters = 0
        while (iters < self.max_iter):
            X_sgd, y_sgd = self.get_batch (X, y, batch_size)
            hypo = g(X_sgd@self.theta)
            err = self.calc_loss(y, hypo)
            neg_gradient = self.neg_gradient(X_sgd, y_sgd, hypo)
            change_norm = self.descent_step (neg_gradient)
            if (change_norm < self.epsilon): 
                break
            iters = iters + 1
        return
    
    # Newton's Method to solve regression (assumed MSE loss function)
    def newton_method (self, X, y):
        """
        ToDo
        Should we make newton_method for other loss functions?
        Add Regularizer?
        """
        g = self.transform_fn_g()
        iters = 0;
        while (iters < self.max_iter):
            hypo = self.g (X @ self.theta)
            neg_gradient = X.T @ (y - hypo)
            hinv = np.linalg.pinv(X.T @ X)
            change = hinv @ neg_gradient
            if (np.linalg.norm (change, ord = 1) < self.epsilon): 
                break
            self.theta = self.theta + change
            iters = iters + 1
        return

    # Normal Equations to solve regression (assumed MSE loss function)
    def normal_eqn (self, X, y):
        """
        ToDo
        Should we make normal equations for other loss functions?
        """
        self.theta = np.linalg.pinv(X.T @ X)@((X.T)@y)
        return

In [601]:
# Get CSV file
def get_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        data = reader(file)
        for row in data:
            if not row:
                continue
            dataset.append(row)
    return dataset

#String to float columnwise
def str_to_float_col(dataset, col):
    for row in dataset:
        row[col] = float(row[col].strip())

def get_csv2 (filename):
    op = np.genfromtxt(filename, delimiter=',')
    op = op[:, 1:]
    return op.astype(np.float)
        
# Split dataset into n folds
def crossval_split(dataset, n_folds):
    split = list()
    dataset_copy = list(dataset)
    fold_dim = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_dim:
            index = random.randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        split.append(fold)
    return split

# Algo evaluation by cross validation split
def eval_algo_cross_val (dataset, n_folds, *args):
    folds = crossval_split(dataset, n_folds)
    mseScores = list()
    maeScores = list()
    rmseScores = list()
    klScores = list()
    jsScores = list()
    ceScores = list()
    r2Scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = fold
        algo = LinearModel() # how to generalize this
        algo.fit(train_set)
        predicted = algo.predict(test_set)
        actual = [row[-1] for row in fold]
        mse = err.mse_calc(actual, predicted)
        mae = err.mae_calc(actual, predicted)
        rmse = err.rmse_calc(actual, predicted)
        #kl_div = err.kl_div_calc(actual, predicted)
        #js_div = err.js_div_calc(actual, predicted)
        #cross_entropy = err.cross_entropy_calc(actual, predicted)
        r2 = err.r2_calc(actual, predicted)
        mseScores.append(mse)
        maeScores.append(mae)
        rmseScores.append(rmse)
        #klScores.append(kl_div)
        #jsScores.append(js_div)
        #ceScores.append(cross_entropy)
        r2Scores.append(r2)
    return mseScores, maeScores, rmseScores, klScores, jsScores, ceScores, r2Scores

# Train over entire set and report training Error
def eval_algo_train (dataset, n_folds, *args):
    #new_d = list ()
    #new_d.append(dataset[0])
    #new_d.append(dataset[1])
    #new_d.append(dataset[2])
    #new_d.append(dataset[3])
    #new_d.append(dataset[4])
    mseScores = list()
    maeScores = list()
    rmseScores = list()
    klScores = list()
    jsScores = list()
    ceScores = list()
    r2Scores = list()
    train_set = list(dataset)
    #test_set = list()
    #for row in dataset:
    #    row_copy = list(row)
    #    test_set.append(row_copy)
    #    row_copy[-1] = None
    algo = LinearModel()
    algo.fit(train_set)
    predicted = algo.predict(train_set)
    actual = [row[-1] for row in dataset]
    mse = err.mse_calc(actual, predicted)
    mae = err.mae_calc(actual, predicted)
    rmse = err.rmse_calc(actual, predicted)
    #kl_div = err.kl_div_calc(actual, predicted)
    #js_div = err.js_div_calc(actual, predicted)
    #cross_entropy = err.cross_entropy_calc(actual, predicted)
    r2 = err.r2_calc(actual, predicted)
    mseScores.append(mse)
    maeScores.append(mae)
    rmseScores.append(rmse)
    #klScores.append(kl_div)
    #jsScores.append(js_div)
    #ceScores.append(cross_entropy)
    r2Scores.append(r2)
    #print (len(predicted), " ", len(actual))
    return mseScores, maeScores, rmseScores, klScores, jsScores, ceScores, r2Scores

# evaluate algorithm
random.seed(1)
filename = 'data/regression.csv'
dataset = get_csv(filename)
dataset.remove(dataset[0]) # remove headings
for i in range(len(dataset[0])): # convert dataset to float columnwise
    str_to_float_col(dataset, i) 
#print (dataset[2])
    
n_folds = 5
print('Running k-fold cross validation with 5 folds')
mseScores, maeScores, rmseScores, klScores, jsScores, ceScores, r2Scores = eval_algo_cross_val (dataset, n_folds)
print('MSE Loss: %s' % mseScores)
print('MAE Loss: %s' % mseScores)
print('RMSE Loss: %s' % rmseScores)
print('R squared Loss: %s' % r2Scores)
print('KL Divergence: %s' % klScores)
print('JS Divergence: %s' % jsScores)
print('Cross Entropy: %s' % ceScores)
print('Mean MSE Loss: %s' % (sum(mseScores)/float(len(mseScores))))
print('Mean MAE Loss: %s' % (sum(mseScores)/float(len(mseScores))))
print('Mean RMSE Loss: %s' % (sum(rmseScores)/float(len(rmseScores))))
print('Mean R squared Loss: %s' % (sum(r2Scores)/float(len(r2Scores))))
#print('Mean KL Divergence: %s' % (sum(klScores)/float(len(klScores))))
#print('Mean JS Divergence: %s' % (sum(jsScores)/float(len(jsScores))))
#print('Mean Cross Entropy: %s' % (sum(ceScores)/float(len(ceScores))))

Running k-fold cross validation with 5 folds


NameError: name 'degrees' is not defined